### Send Function in LangGraph
The Send function allows you to specify the target node and the message (data or state) to pass along. It’s designed to streamline communication between nodes in a graph. The Send function is essential for orchestrating data flow in LangGraph.

The Send function simplifies complex workflows by:
* Dynamically routing data between nodes.
* Supporting parallel processing for tasks like batch operations.
* Keeping your workflow modular and easy to debug.

Send(target_node: str, message: dict)
* target_node: The name of the node receiving the message.
* message: A dictionary containing the state or data to be sent.

In [31]:
from typing import TypedDict, Annotated
import operator

class OverallState(TypedDict):
    subjects: list[str]
    jokes: Annotated[list[str], operator.add]  # Fixed indentation


In [62]:
import time
from langgraph.constants import Send

def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": s}) for s in state['subjects']]

def generate_joke(state: OverallState):
    subject = state['subject']
    print(f"Generating joke for subject: {subject}")
    joke = f"Why did the {subject} cross the road? To get to the other side!"
    time.sleep(5)
    print(f"Done with time for {subject}")
    return {"jokes": [joke]}

In [63]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(OverallState)
builder.add_node("continue_to_jokes", continue_to_jokes)
builder.add_node("generate_joke", generate_joke)
builder.add_conditional_edges(START, continue_to_jokes)
builder.add_edge("generate_joke", END)
app = builder.compile()


In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

In [65]:
initial_state = {"subjects": ["chicken", "cow"]}
result = app.invoke(initial_state)
print(result)

Generating joke for subject: chickenGenerating joke for subject: cow

Done with time for chickenDone with time for cow

{'subjects': ['chicken', 'cow'], 'jokes': ['Why did the chicken cross the road? To get to the other side!', 'Why did the cow cross the road? To get to the other side!']}


{'subjects': ['chicken', 'cow'], 
'jokes': ['Why did the chicken cross the road? To get to the other side!', 'Why did the cow cross the road? To get to the other side!']}